To implement the encodeRS(m) function for a [255, 128] Reed Solomon code with generator polynomial g(x) = (x-1)(x-α)(x-α^2)*(x-α^3)...(x-α^127), we can use the following steps:

    Convert the input message m into a polynomial m(x) of degree k-1, where k = 128, by setting m(x) = m_0 + m_1x + ... + m_k-1x^(k-1).

    Pad the message polynomial m(x) with k = 128 zeros to form a new polynomial M(x) of degree n-1 = 254, where n = 255. That is, set M(x) = m(x)*x^k.

    Calculate the remainder polynomial R(x) = M(x) mod g(x) using polynomial long division or the Euclidean algorithm in GF(2^8).

    The codeword c can be formed by appending the coefficients of the remainder polynomial R(x) to the message polynomial m(x), i.e., set c(x) = m(x) + R(x), and then evaluate the resulting polynomial c(x) at each element of GF(2^8) to obtain the codeword c = (c_0, c_1, ..., c_n-1).

In [145]:
# d = floor(128-1 / 2) = 63 
# i.e correct up to 63 errors 


In [146]:
import secrets
q = 2 
m = 8 
p = x^8 + x^4 + x^3 + x^2 + 1
F.<a> = GF(q^m, modulus=p) 
Fqm.<y> = GF(q^m) 
R.<x> = PolynomialRing(F) 

n = 255
k = 128 

message_length = 128 # bytes 

In [147]:
def text2bin(msg):
    return "".join([f"{ord(m):08b}" for m in msg])

def bin2text(bin_s):
    return "".join([chr(int(bin_s[i: i+8], 2)) for i in range(0, len(bin_s), 8)])

def to_field_element(string):
    """Binary String to field Element"""
    elem = 0
    n = len(string)
    for i in range(n):
        coeff = int(string[n-i-1])
        elem += coeff * a^i # coeff is 0 or 1
    return elem


def from_field_element(field_element):
    """Field Element to binary sring"""
    return ''.join(map(str, vector(field_element)[::-1]))

def convert_str_to_polynomial(msg): 
    msg = msg + "x"*(k - len(msg)) 
    # generate random message 
    # message = [random.randint(0, 255) for _ in range(message_length)]
    message = text2bin(msg)
    message = [message[i:i+8] for i in range(0, k, 8)]
    
    message = sum([to_field_element(e)*x^i for i,e in enumerate(message)])
    
    return message

In [148]:
# define generator poly 
g_x = sum([(x - (a^i)) for i in range(q^m / 2 - 1)])
# degree of generator poly. should be le #codewords
g_x

x + a^7 + a^5 + a^4

In [168]:

def encodeRS(g, m): 
    """
    Takes a 128 - byte messagee, and encode it to a codeword 
        c, with a systematic encoding as 
        
        m = (m_0,m_1,...,m_k-1) => c = (m_0, m_1,...,m_k-1, c_k,..._c_n-1)
    """
    m_x = convert_str_to_polynomial(m)
    # Now perform encoding 
    # c(x) => m(x) * x^(n-k) % g(x) 

    # formula book : 
    # m(x)*x^(n-k) - Remainder(m(x)*x^(n-k), g(x)) 
    
    tmp = m_x * x^(n-k) 
    _, rem = tmp.quo_rem(g)
    c_x = tmp - rem
    print(c_x) 
    print() 
    c_x = c_x
    return c_x



In [169]:
text = "This is a sample text, that is not too long"  

encodeRS(g_x, text) 

(a^6 + a^5 + a^2 + 1)*x^142 + (a^6 + a^5 + a^3 + a^2)*x^141 + (a^6 + a^5 + a^4)*x^140 + (a^6 + a^5 + a^3 + a^2 + 1)*x^139 + (a^6 + a^5 + 1)*x^138 + (a^6 + a^5 + a^4 + a + 1)*x^137 + a^5*x^136 + (a^6 + a^5 + 1)*x^135 + a^5*x^134 + (a^6 + a^5 + a^4 + a + 1)*x^133 + (a^6 + a^5 + a^3 + 1)*x^132 + a^5*x^131 + (a^6 + a^5 + a^4 + a + 1)*x^130 + (a^6 + a^5 + a^3 + 1)*x^129 + (a^6 + a^5 + a^3)*x^128 + (a^6 + a^4 + a^2)*x^127 + a^7 + a^5 + a^4 + a^3 + a^2 + a



(a^6 + a^5 + a^2 + 1)*x^142 + (a^6 + a^5 + a^3 + a^2)*x^141 + (a^6 + a^5 + a^4)*x^140 + (a^6 + a^5 + a^3 + a^2 + 1)*x^139 + (a^6 + a^5 + 1)*x^138 + (a^6 + a^5 + a^4 + a + 1)*x^137 + a^5*x^136 + (a^6 + a^5 + 1)*x^135 + a^5*x^134 + (a^6 + a^5 + a^4 + a + 1)*x^133 + (a^6 + a^5 + a^3 + 1)*x^132 + a^5*x^131 + (a^6 + a^5 + a^4 + a + 1)*x^130 + (a^6 + a^5 + a^3 + 1)*x^129 + (a^6 + a^5 + a^3)*x^128 + (a^6 + a^4 + a^2)*x^127 + a^7 + a^5 + a^4 + a^3 + a^2 + a